In [86]:
using ModelingToolkit, OrdinaryDiffEq, Plots
using ModelingToolkit: t_nounits as t, D_nounits as d

### Models

In [87]:
@connector Pin begin
    v(t) # Potential at the pin
    i(t), [connect = Flow] # Current flowing into the pin
end

@mtkmodel OnePort begin
    @components begin
        p = Pin()
        n = Pin()
    end
    @variables begin
        v(t) # Voltage drop of the two pins (= p.v - n.v)
        i(t) # Current flowing into the pin
    end
    @equations begin
        v ~ p.v - n.v
        0 ~ p.i + n.i
        i ~ p.i
    end
end

ModelingToolkit.Model{typeof(__OnePort__), Dict{Symbol, Any}}(__OnePort__, Dict{Symbol, Any}(:components => Any[Union{Expr, Symbol}[:p, :Pin], Union{Expr, Symbol}[:n, :Pin]], :variables => Dict{Symbol, Dict{Symbol, Any}}(:v => Dict(:type => Real), :i => Dict(:type => Real)), :kwargs => Dict{Symbol, Dict}(:v => Dict{Symbol, Union{Nothing, DataType}}(:value => nothing, :type => Real), :i => Dict{Symbol, Union{Nothing, DataType}}(:value => nothing, :type => Real)), :independent_variable => :t, :equations => Any["v ~ p.v - n.v", "v ~ p.v - n.v", "0 ~ p.i + n.i", "v ~ p.v - n.v", "0 ~ p.i + n.i", "i ~ p.i"]), false)

In [88]:
@connector HeatPort begin
    T(t)
    Q_flow(t), [connect = Flow]
end

@connector HeatPort_a begin
    @extend T, Q_flow = heatPort_a = HeatPort()
end

@connector HeatPort_b begin
    @extend T, Q_flow = heatPort_b = HeatPort()
end

ModelingToolkit.Model{typeof(__HeatPort_b__), Dict{Symbol, Any}}(__HeatPort_b__, Dict{Symbol, Any}(:independent_variable => t, :extend => Any[[:T, :Q_flow], :heatPort_b, :HeatPort]), true)

In [89]:
@mtkmodel FixedTemperature begin
    @components begin
        port = HeatPort_b()
    end
    @parameters begin
        T, [description = "Fixed temperature boundary condition"]
    end
    @equations begin
        port.T ~ T
    end
end

ModelingToolkit.Model{typeof(__FixedTemperature__), Dict{Symbol, Any}}(__FixedTemperature__, Dict{Symbol, Any}(:components => Any[Union{Expr, Symbol}[:port, :HeatPort_b]], :kwargs => Dict{Symbol, Dict}(:T => Dict{Symbol, Union{Nothing, DataType}}(:value => nothing, :type => Real)), :independent_variable => t, :parameters => Dict{Symbol, Dict{Symbol, Any}}(:T => Dict(:type => Real, :description => "Fixed temperature boundary condition")), :equations => Any["port.T ~ T"]), false)

In [90]:
# copy from ModelingToolkitStandardLibrary
@mtkmodel HeatingResistor begin
    @extend v, i = oneport = OnePort()
    @components begin
        heat_port = HeatPort()
    end
    @parameters begin
        R_ref = 1.0, [description = "Reference resistance"]
        T_ref = 300.15, [description = "Reference temperature"]
        alpha = 0, [description = "Temperature coefficient of resistance"]
    end
    @variables begin
        R(t)#, [guess = R_ref]
    end
    @equations begin
        R ~ R_ref * (1 + alpha * (heat_port.T - T_ref))
        heat_port.Q_flow ~ -v * i # -LossPower
        v ~ i * R 
        # ExtraVariablesSystemException: The system is unbalanced. 
        # There are 25 highest order derivative variables and 24 equations.
    end
end

ModelingToolkit.Model{typeof(__HeatingResistor__), Dict{Symbol, Any}}(__HeatingResistor__, Dict{Symbol, Any}(:equations => Any["R ~ R_ref * (1 + alpha * (heat_port.T - T_ref))", "R ~ R_ref * (1 + alpha * (heat_port.T - T_ref))", "heat_port.Q_flow ~ -v * i", "R ~ R_ref * (1 + alpha * (heat_port.T - T_ref))", "heat_port.Q_flow ~ -v * i", "v ~ i * R"], :components => Any[Union{Expr, Symbol}[:heat_port, :HeatPort]], :variables => Dict{Symbol, Dict{Symbol, Any}}(:R => Dict(:type => Real)), :kwargs => Dict{Symbol, Dict}(:alpha => Dict{Symbol, Any}(:value => 0, :type => Real), :T_ref => Dict{Symbol, Any}(:value => 300.15, :type => Real), :R => Dict{Symbol, Union{Nothing, DataType}}(:value => nothing, :type => Real), :R_ref => Dict{Symbol, Any}(:value => 1.0, :type => Real)), :independent_variable => :t, :extend => Any[[:v, :i], :oneport, :OnePort], :parameters => Dict{Symbol, Dict{Symbol, Any}}(:alpha => Dict(:default => 0, :type => Real, :description => "Temperature coefficient of resistance

In [91]:
@mtkmodel Ground begin
    @components begin
        p = Pin()
    end
    @equations begin
        p.v ~ 0
    end
end

@mtkmodel Capacitor begin
    @extend i,v = oneport = OnePort()
    @parameters begin
        C # Capacitance
    end
    @equations begin
        i ~ C * d(v)
    end
end

@mtkmodel ConstantVoltage begin
    @extend v, = oneport = OnePort()    
    @parameters begin
        V # Value of constant voltage
    end
    @equations begin
        v ~ V
    end
end

ModelingToolkit.Model{typeof(__ConstantVoltage__), Dict{Symbol, Any}}(__ConstantVoltage__, Dict{Symbol, Any}(:equations => Any["v ~ V"], :kwargs => Dict{Symbol, Dict}(:V => Dict{Symbol, Union{Nothing, DataType}}(:value => nothing, :type => Real)), :independent_variable => t, :extend => Any[[:v], :oneport, :OnePort], :parameters => Dict{Symbol, Dict{Symbol, Any}}(:V => Dict(:type => Real))), false)

In [92]:
@mtkmodel RcHeating begin    
    @components begin
        resistor = HeatingResistor()           
        capacitor = Capacitor()
        source = ConstantVoltage()
        ground = Ground()
        fixedTemperature = FixedTemperature()
        
    end      
    @equations begin        
        connect(source.p, resistor.p)
        connect(resistor.n, capacitor.p)
        connect(capacitor.n, source.n, ground.p) 
        connect(resistor.heat_port, fixedTemperature.port)       
    end
end

ModelingToolkit.Model{typeof(__RcHeating__), Dict{Symbol, Any}}(__RcHeating__, Dict{Symbol, Any}(:components => Any[Union{Expr, Symbol}[:resistor, :HeatingResistor], Union{Expr, Symbol}[:capacitor, :Capacitor], Union{Expr, Symbol}[:source, :ConstantVoltage], Union{Expr, Symbol}[:ground, :Ground], Union{Expr, Symbol}[:fixedTemperature, :FixedTemperature]], :independent_variable => t, :equations => Any["connect(source.p, resistor.p)", "connect(source.p, resistor.p)", "connect(resistor.n, capacitor.p)", "connect(source.p, resistor.p)", "connect(resistor.n, capacitor.p)", "connect(capacitor.n, source.n, ground.p)", "connect(source.p, resistor.p)", "connect(resistor.n, capacitor.p)", "connect(capacitor.n, source.n, ground.p)", "connect(resistor.heat_port, fixedTemperature.port)"]), false)

### 0. sys

In [93]:
@mtkbuild sys = RcHeating()

Model sys:
Equations (2):
  2 standard: see equations(sys)
Unknowns (2): see unknowns(sys)
  capacitor₊v(t)
  resistor₊i(t)
Parameters (6): see parameters(sys)
  resistor₊alpha [defaults to 0]: Temperature coefficient of resistance
  fixedTemperature₊T: Fixed temperature boundary condition
  source₊V
  resistor₊R_ref [defaults to 1.0]: Reference resistance
  ⋮
Observed (23): see observed(sys)

In [107]:
"""
Unknowns (2)
    capacitor₊v(t) ▪ Unassigned ▪ Voltage drop of the two pins (= p.v - n.v)
    resistor₊i(t) ▪ Unassigned ▪ Current flowing from pin p to pin n
Parameters (6)
    resistor₊alpha ▪ 0 ▪ Temperature coefficient of resistance
    fixedTemperature₊T ▪ Unassigned ▪ Fixed temperature at port
    source₊V ▪ Unassigned ▪ Value of constant voltage
    resistor₊R_ref ▪ 1.0 ▪ Reference resistance
    resistor₊T_ref ▪ 300.15 ▪ Reference temperature
    capacitor₊C ▪ Unassigned ▪ Capacitance
Equations (2)
    Differential(t)(capacitor₊v(t)) ~ capacitor₊i(t) / capacitor₊C
    0 ~ -resistor₊v(t) + resistor₊i(t)*resistor₊R(t)
"""
nothing

Core.nothing

In [95]:
parameters(sys) |> display
observed(sys) |> display
unknowns(sys) |> display
equations(sys) |> display

6-element Vector{Any}:
 resistor₊alpha
 fixedTemperature₊T
 source₊V
 resistor₊R_ref
 resistor₊T_ref
 capacitor₊C

23-element Vector{Equation}:
 capacitor₊p₊v(t) ~ capacitor₊v(t)
 source₊v(t) ~ source₊V
 resistor₊p₊i(t) ~ resistor₊i(t)
 fixedTemperature₊port₊T(t) ~ fixedTemperature₊T
 capacitor₊i(t) ~ resistor₊i(t)
 source₊i(t) ~ -resistor₊i(t)
 ground₊p₊v(t) ~ 0.0
 capacitor₊n₊v(t) ~ -0.0
 resistor₊n₊v(t) ~ capacitor₊p₊v(t)
 resistor₊v(t) ~ -capacitor₊v(t) + source₊v(t)
 ⋮
 source₊p₊i(t) ~ source₊i(t)
 source₊n₊v(t) ~ capacitor₊n₊v(t)
 source₊p₊v(t) ~ capacitor₊p₊v(t) + resistor₊v(t)
 resistor₊heat_port₊Q_flow(t) ~ -resistor₊i(t)*resistor₊v(t)
 resistor₊R(t) ~ -resistor₊R_ref*(-1 - (-resistor₊T_ref + resistor₊heat_port₊T(t))*resistor₊alpha)
 capacitor₊n₊i(t) ~ -capacitor₊p₊i(t)
 source₊n₊i(t) ~ -source₊p₊i(t)
 resistor₊p₊v(t) ~ source₊p₊v(t)
 fixedTemperature₊port₊Q_flow(t) ~ -resistor₊heat_port₊Q_flow(t)

2-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 capacitor₊v(t)
 resistor₊i(t)

2-element Vector{Equation}:
 Differential(t)(capacitor₊v(t)) ~ capacitor₊i(t) / capacitor₊C
 0 ~ -resistor₊v(t) + resistor₊i(t)*resistor₊R(t)

### .1 (sys)

In [96]:
function display_isys(isys)
    # isys_ps = parameters(isys)
    isys_eqs = equations(isys)
    isys_unks = unknowns(isys)
    isys_obs = observed(isys)

    # display("$(length(isys_ps)) parameters")
    # display(isys_ps)
    display("$(length(isys_eqs)) equations")
    display(isys_eqs)
    display("$(length(isys_unks)) unknowns")
    display(isys_unks)
    display("$(length(isys_obs)) observed")
    display(isys_obs)
end

display_isys (generic function with 1 method)

In [97]:
isys1 = generate_initializesystem(sys)
display_isys(isys1)

"24 equations"

24-element Vector{Equation}:
 0 ~ -resistor₊v(t) + resistor₊i(t)*resistor₊R(t)
 0 ~ capacitor₊v(t) - capacitor₊p₊v(t)
 0 ~ source₊V - source₊v(t)
 0 ~ resistor₊i(t) - resistor₊p₊i(t)
 0 ~ fixedTemperature₊T - fixedTemperature₊port₊T(t)
 0 ~ resistor₊i(t) - capacitor₊i(t)
 0 ~ -source₊i(t) - resistor₊i(t)
 0 ~ -ground₊p₊v(t)
 0 ~ -capacitor₊n₊v(t)
 0 ~ -resistor₊n₊v(t) + capacitor₊p₊v(t)
 ⋮
 0 ~ source₊i(t) - source₊p₊i(t)
 0 ~ -source₊n₊v(t) + capacitor₊n₊v(t)
 0 ~ capacitor₊p₊v(t) + resistor₊v(t) - source₊p₊v(t)
 0 ~ -resistor₊heat_port₊Q_flow(t) - resistor₊i(t)*resistor₊v(t)
 0 ~ -resistor₊R(t) - resistor₊R_ref*(-1 - (-resistor₊T_ref + resistor₊heat_port₊T(t))*resistor₊alpha)
 0 ~ -capacitor₊p₊i(t) - capacitor₊n₊i(t)
 0 ~ -source₊p₊i(t) - source₊n₊i(t)
 0 ~ source₊p₊v(t) - resistor₊p₊v(t)
 0 ~ -fixedTemperature₊port₊Q_flow(t) - resistor₊heat_port₊Q_flow(t)

"25 unknowns"

25-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 capacitor₊v(t)
 resistor₊i(t)
 capacitor₊p₊v(t)
 source₊v(t)
 resistor₊p₊i(t)
 fixedTemperature₊port₊T(t)
 capacitor₊i(t)
 source₊i(t)
 ground₊p₊v(t)
 capacitor₊n₊v(t)
 ⋮
 source₊p₊i(t)
 source₊n₊v(t)
 source₊p₊v(t)
 resistor₊heat_port₊Q_flow(t)
 resistor₊R(t)
 capacitor₊n₊i(t)
 source₊n₊i(t)
 resistor₊p₊v(t)
 fixedTemperature₊port₊Q_flow(t)

"0 observed"

Equation[]

In [98]:
isys1_prob = ODEProblem(sys, [], (0.0, 10.0))
# Warning: Did not converge after `maxiters = 100` substitutions

┌ Warning: Initialization system is underdetermined. 0 equations for 1 unknowns. Initialization will default to using least squares. `SCCNonlinearProblem` can only be used for initialization of fully determined systems and hence will not be used here. To suppress this warning pass warn_initialize_determined = false. To make this warning into an error, pass fully_determined = true
└ @ ModelingToolkit C:\Users\TCY\.julia\packages\ModelingToolkit\aau6A\src\systems\diffeqs\abstractodesystem.jl:1523
┌ Warning: Did not converge after `maxiters = 100` substitutions. Either there is a cycle in the rules or `maxiters` needs to be higher.
└ @ Symbolics C:\Users\TCY\.julia\packages\Symbolics\tSVcV\src\variable.jl:587


ModelingToolkit.MissingGuessError: Cyclic guesses detected in the system. Symbolic values were found for the following variables/parameters in the map: 
resistor₊p₊i(t)  => resistor₊i(t)
In order to resolve this, please provide additional numeric guesses so that the chain can be resolved to assign numeric values to each variable.            


### .2 (sys; u0map, pmap)

In [99]:
isys2 = generate_initializesystem(sys; 
            u0map = [sys.capacitor.v => 0.0, sys.resistor.i => 0.0],
            pmap = [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15])
display_isys(isys2)

"26 equations"

26-element Vector{Equation}:
 0 ~ -resistor₊v(t) + resistor₊i(t)*resistor₊R(t)
 0 ~ -capacitor₊v(t)
 0 ~ -resistor₊i(t)
 0 ~ capacitor₊v(t) - capacitor₊p₊v(t)
 0 ~ source₊V - source₊v(t)
 0 ~ resistor₊i(t) - resistor₊p₊i(t)
 0 ~ fixedTemperature₊T - fixedTemperature₊port₊T(t)
 0 ~ resistor₊i(t) - capacitor₊i(t)
 0 ~ -source₊i(t) - resistor₊i(t)
 0 ~ -ground₊p₊v(t)
 ⋮
 0 ~ source₊i(t) - source₊p₊i(t)
 0 ~ -source₊n₊v(t) + capacitor₊n₊v(t)
 0 ~ capacitor₊p₊v(t) + resistor₊v(t) - source₊p₊v(t)
 0 ~ -resistor₊heat_port₊Q_flow(t) - resistor₊i(t)*resistor₊v(t)
 0 ~ -resistor₊R(t) - resistor₊R_ref*(-1 - (-resistor₊T_ref + resistor₊heat_port₊T(t))*resistor₊alpha)
 0 ~ -capacitor₊p₊i(t) - capacitor₊n₊i(t)
 0 ~ -source₊p₊i(t) - source₊n₊i(t)
 0 ~ source₊p₊v(t) - resistor₊p₊v(t)
 0 ~ -fixedTemperature₊port₊Q_flow(t) - resistor₊heat_port₊Q_flow(t)

"25 unknowns"

25-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 capacitor₊v(t)
 resistor₊i(t)
 capacitor₊p₊v(t)
 source₊v(t)
 resistor₊p₊i(t)
 fixedTemperature₊port₊T(t)
 capacitor₊i(t)
 source₊i(t)
 ground₊p₊v(t)
 capacitor₊n₊v(t)
 ⋮
 source₊p₊i(t)
 source₊n₊v(t)
 source₊p₊v(t)
 resistor₊heat_port₊Q_flow(t)
 resistor₊R(t)
 capacitor₊n₊i(t)
 source₊n₊i(t)
 resistor₊p₊v(t)
 fixedTemperature₊port₊Q_flow(t)

"0 observed"

Equation[]

In [100]:
isys2_prob = ODEProblem(sys, [sys.capacitor.v => 0.0, sys.resistor.i => 0.0], (0.0, 10.0),
                [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15])

┌ Warning: Initialization system is overdetermined. 1 equations for 0 unknowns. Initialization will default to using least squares. `SCCNonlinearProblem` can only be used for initialization of fully determined systems and hence will not be used here. To suppress this warning pass warn_initialize_determined = false. To make this warning into an error, pass fully_determined = true
└ @ ModelingToolkit C:\Users\TCY\.julia\packages\ModelingToolkit\aau6A\src\systems\diffeqs\abstractodesystem.jl:1520


ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
Initialization status: OVERDETERMINED
Non-trivial mass matrix: true
timespan: (0.0, 10.0)
u0: 2-element Vector{Float64}:
 0.0
 0.0

In [101]:
isys2_prob_initsys = isys2_prob.f.initializeprob.f.sys

Model sys:
Equations (1):
  1 standard: see equations(sys)
Parameters (57): see parameters(sys)
  resistor₊alpha [defaults to 0]: Temperature coefficient of resistance
  fixedTemperature₊T: Fixed temperature boundary condition
  t
  source₊V
  ⋮
Observed (25): see observed(sys)

In [102]:
structural_simplify(isys2; fully_determined = false)

Model sys:
Equations (1):
  1 standard: see equations(sys)
Parameters (57): see parameters(sys)
  resistor₊alpha [defaults to 0]: Temperature coefficient of resistance
  fixedTemperature₊T: Fixed temperature boundary condition
  t
  source₊V
  ⋮
Observed (25): see observed(sys)

### .3 (sys; u0manp, pmap, guesses)

In [103]:
isys3 = generate_initializesystem(sys; 
            u0map = [sys.capacitor.v => 0.0, sys.resistor.i => 0.0],
            pmap = [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15],
            guesses = [sys.resistor.R=>sys.resistor.R_ref])
display_isys(isys3)

"26 equations"

26-element Vector{Equation}:
 0 ~ -resistor₊v(t) + resistor₊i(t)*resistor₊R(t)
 0 ~ -capacitor₊v(t)
 0 ~ -resistor₊i(t)
 0 ~ capacitor₊v(t) - capacitor₊p₊v(t)
 0 ~ source₊V - source₊v(t)
 0 ~ resistor₊i(t) - resistor₊p₊i(t)
 0 ~ fixedTemperature₊T - fixedTemperature₊port₊T(t)
 0 ~ resistor₊i(t) - capacitor₊i(t)
 0 ~ -source₊i(t) - resistor₊i(t)
 0 ~ -ground₊p₊v(t)
 ⋮
 0 ~ source₊i(t) - source₊p₊i(t)
 0 ~ -source₊n₊v(t) + capacitor₊n₊v(t)
 0 ~ capacitor₊p₊v(t) + resistor₊v(t) - source₊p₊v(t)
 0 ~ -resistor₊heat_port₊Q_flow(t) - resistor₊i(t)*resistor₊v(t)
 0 ~ -resistor₊R(t) - resistor₊R_ref*(-1 - (-resistor₊T_ref + resistor₊heat_port₊T(t))*resistor₊alpha)
 0 ~ -capacitor₊p₊i(t) - capacitor₊n₊i(t)
 0 ~ -source₊p₊i(t) - source₊n₊i(t)
 0 ~ source₊p₊v(t) - resistor₊p₊v(t)
 0 ~ -fixedTemperature₊port₊Q_flow(t) - resistor₊heat_port₊Q_flow(t)

"25 unknowns"

25-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 capacitor₊v(t)
 resistor₊i(t)
 capacitor₊p₊v(t)
 source₊v(t)
 resistor₊p₊i(t)
 fixedTemperature₊port₊T(t)
 capacitor₊i(t)
 source₊i(t)
 ground₊p₊v(t)
 capacitor₊n₊v(t)
 ⋮
 source₊p₊i(t)
 source₊n₊v(t)
 source₊p₊v(t)
 resistor₊heat_port₊Q_flow(t)
 resistor₊R(t)
 capacitor₊n₊i(t)
 source₊n₊i(t)
 resistor₊p₊v(t)
 fixedTemperature₊port₊Q_flow(t)

"0 observed"

Equation[]

In [104]:
isys3_prob = ODEProblem(sys, [sys.capacitor.v => 0.0, sys.resistor.i => 0.0], (0.0, 10.0),
                [sys.capacitor.C => 1.0, sys.source.V=>1.0,sys.fixedTemperature.T=>400.15],
                guesses = [sys.resistor.R=>sys.resistor.R_ref])

┌ Warning: Initialization system is overdetermined. 1 equations for 0 unknowns. Initialization will default to using least squares. `SCCNonlinearProblem` can only be used for initialization of fully determined systems and hence will not be used here. To suppress this warning pass warn_initialize_determined = false. To make this warning into an error, pass fully_determined = true
└ @ ModelingToolkit C:\Users\TCY\.julia\packages\ModelingToolkit\aau6A\src\systems\diffeqs\abstractodesystem.jl:1520


ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
Initialization status: OVERDETERMINED
Non-trivial mass matrix: true
timespan: (0.0, 10.0)
u0: 2-element Vector{Float64}:
 0.0
 0.0

In [105]:
isys3_prob_initsys = isys3_prob.f.initializeprob.f.sys

Model sys:
Equations (1):
  1 standard: see equations(sys)
Parameters (57): see parameters(sys)
  resistor₊alpha [defaults to 0]: Temperature coefficient of resistance
  fixedTemperature₊T: Fixed temperature boundary condition
  t
  source₊V
  ⋮
Observed (25): see observed(sys)

In [106]:
structural_simplify(isys3; fully_determined = false)

Model sys:
Equations (1):
  1 standard: see equations(sys)
Parameters (57): see parameters(sys)
  resistor₊alpha [defaults to 0]: Temperature coefficient of resistance
  fixedTemperature₊T: Fixed temperature boundary condition
  t
  source₊V
  ⋮
Observed (25): see observed(sys)